In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from evaluation import ModelEvaluator
from data_preprocessing import MovieDataPreprocessor

# Cargar datos de prueba
DATA_PATH = "../data/raw"
preprocessor = MovieDataPreprocessor(DATA_PATH)
X, y_labels = preprocessor.load_data()
y_categorical, y_encoded = preprocessor.preprocess_labels(y_labels)
X_train, X_val, X_test, y_train, y_val, y_test = preprocessor.create_data_splits(X, y_categorical)

# Cargar modelos entrenados
cnn_model = load_model('../data/models/cnn_best.h5')
mlp_model = load_model('../data/models/mlp_best.h5')

print("✅ Modelos y datos cargados correctamente")

# Celda 2: Evaluación detallada
evaluator = ModelEvaluator(preprocessor.label_encoder)

print("📊 EVALUACIÓN DETALLADA DE MODELOS")
print("="*45)

# Evaluar CNN
cnn_metrics, cnn_report, cnn_pred, cnn_proba = evaluator.evaluate_model(cnn_model, X_test, y_test)
print("\n🔥 Resultados CNN:")
for metric, value in cnn_metrics.items():
    print(f"  {metric.title()}: {value:.4f}")

# Evaluar MLP
mlp_metrics, mlp_report, mlp_pred, mlp_proba = evaluator.evaluate_model(mlp_model, X_test, y_test)
print("\n🧠 Resultados MLP:")
for metric, value in mlp_metrics.items():
    print(f"  {metric.title()}: {value:.4f}")

# Celda 3: Análisis de errores
def analyze_prediction_errors(y_true, y_pred, y_proba, model_name):
    """Analizar errores de predicción"""
    print(f"\n🔍 ANÁLISIS DE ERRORES - {model_name}")
    print("-" * 40)
    
    # Encontrar predicciones incorrectas
    wrong_predictions = np.where(y_true != y_pred)[0]
    confidence_scores = np.max(y_proba, axis=1)
    
    print(f"Predicciones incorrectas: {len(wrong_predictions)}/{len(y_true)} ({len(wrong_predictions)/len(y_true)*100:.1f}%)")
    
    # Errores con alta confianza (falsos positivos seguros)
    confident_errors = wrong_predictions[confidence_scores[wrong_predictions] > 0.8]
    print(f"Errores con alta confianza (>80%): {len(confident_errors)}")
    
    # Errores con baja confianza
    unconfident_errors = wrong_predictions[confidence_scores[wrong_predictions] < 0.5]
    print(f"Errores con baja confianza (<50%): {len(unconfident_errors)}")
    
    return wrong_predictions, confidence_scores

y_true = np.argmax(y_test, axis=1)
cnn_errors, cnn_confidence = analyze_prediction_errors(y_true, cnn_pred, cnn_proba, "CNN")
mlp_errors, mlp_confidence = analyze_prediction_errors(y_true, mlp_pred, mlp_proba, "MLP")

# Celda 4: Visualización de predicciones incorrectas
def plot_wrong_predictions(X_test, y_true, y_pred, y_proba, label_encoder, errors, title, n_samples=8):
    """Visualizar predicciones incorrectas"""
    selected_errors = np.random.choice(errors, min(n_samples, len(errors)), replace=False)
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    for i, idx in enumerate(selected_errors):
        ax = axes[i]
        ax.imshow(X_test[idx])
        
        true_label = label_encoder.inverse_transform([y_true[idx]])[0]
        pred_label = label_encoder.inverse_transform([y_pred[idx]])[0]
        confidence = np.max(y_proba[idx]) * 100
        
        ax.set_title(f'Real: {true_label}\nPredicción: {pred_label}\nConfianza: {confidence:.1f}%', 
                    fontsize=10)
        ax.axis('off')
    
    plt.suptitle(f'{title} - Predicciones Incorrectas', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

if len(cnn_errors) > 0:
    plot_wrong_predictions(X_test, y_true, cnn_pred, cnn_proba, 
                          preprocessor.label_encoder, cnn_errors, "CNN")

if len(mlp_errors) > 0:
    plot_wrong_predictions(X_test, y_true, mlp_pred, mlp_proba, 
                          preprocessor.label_encoder, mlp_errors, "MLP")

# Celda 5: Análisis por clase
def analyze_per_class_performance(y_true, y_pred, label_encoder):
    """Análisis de rendimiento por clase"""
    from sklearn.metrics import classification_report
    
    report = classification_report(y_true, y_pred, 
                                 target_names=label_encoder.classes_, 
                                 output_dict=True)
    
    # Convertir a DataFrame
    df_report = pd.DataFrame(report).transpose()
    df_report = df_report.iloc[:-3, :-1]  # Remover resumen
    
    return df_report

print("\n📈 RENDIMIENTO POR CLASE")
print("="*30)

cnn_class_perf = analyze_per_class_performance(y_true, cnn_pred, preprocessor.label_encoder)
mlp_class_perf = analyze_per_class_performance(y_true, mlp_pred, preprocessor.label_encoder)

print("\nCNN - Rendimiento por clase:")
print(cnn_class_perf.round(3))

print("\nMLP - Rendimiento por clase:")
print(mlp_class_perf.round(3))

# Celda 6: Visualización comparativa final
def create_final_comparison_plot(cnn_metrics, mlp_metrics):
    """Crear gráfico comparativo final"""
    metrics = list(cnn_metrics.keys())
    cnn_values = list(cnn_metrics.values())
    mlp_values = list(mlp_metrics.values())
    
    x = np.arange(len(metrics))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bars1 = ax.bar(x - width/2, cnn_values, width, label='CNN', color='skyblue', alpha=0.8)
    bars2 = ax.bar(x + width/2, mlp_values, width, label='MLP', color='lightcoral', alpha=0.8)
    
    ax.set_xlabel('Métricas')
    ax.set_ylabel('Score')
    ax.set_title('Comparación Final de Modelos')
    ax.set_xticks(x)
    ax.set_xticklabels([m.title() for m in metrics])
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Agregar valores en las barras
    def autolabel(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.3f}',
                       xy=(bar.get_x() + bar.get_width() / 2, height),
                       xytext=(0, 3),
                       textcoords="offset points",
                       ha='center', va='bottom')
    
    autolabel(bars1)
    autolabel(bars2)
    
    plt.tight_layout()
    plt.savefig('../results/plots/final_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

create_final_comparison_plot(cnn_metrics, mlp_metrics)

# Celda 7: Generación de reporte
def generate_summary_report(cnn_metrics, mlp_metrics, dataset_info):
    """Generar reporte resumen"""
    print("\n" + "="*60)
    print("REPORTE FINAL DEL PROYECTO")
    print("="*60)
    
    print(f"\n📊 INFORMACIÓN DEL DATASET:")
    print(f"  • Total de muestras: {dataset_info['total_samples']}")
    print(f"  • Número de clases: {dataset_info['num_classes']}")
    print(f"  • Clases: {', '.join(dataset_info['class_distribution'].keys())}")
    
    print(f"\n🏆 RESULTADOS FINALES:")
    print(f"  CNN:")
    for metric, value in cnn_metrics.items():
        print(f"    • {metric.title()}: {value:.4f}")
    
    print(f"  MLP:")
    for metric, value in mlp_metrics.items():
        print(f"    • {metric.title()}: {value:.4f}")
    
    # Determinar mejor modelo
    mejor_modelo = 'CNN' if cnn_metrics['accuracy'] > mlp_metrics['accuracy'] else 'MLP'
    diferencia = abs(cnn_metrics['accuracy'] - mlp_metrics['accuracy'])
    
    print(f"\n🥇 MEJOR MODELO: {mejor_modelo}")
    print(f"   Diferencia en accuracy: {diferencia:.4f} ({diferencia*100:.2f}%)")
    
    print(f"\n💡 CONCLUSIONES:")
    if diferencia < 0.05:
        print("   • Los modelos tienen rendimiento similar")
        print("   • La diferencia es menor al 5%")
    else:
        print(f"   • El modelo {mejor_modelo} supera significativamente al otro")
    
    if cnn_metrics['accuracy'] > mlp_metrics['accuracy']:
        print("   • Las CNNs son más efectivas para análisis de imágenes")
        print("   • La extracción de características espaciales es clave")
    else:
        print("   • El MLP logró competir con la CNN")
        print("   • Posible overfitting en la CNN o underfitting")

dataset_info = preprocessor.get_dataset_info(y_categorical)
generate_summary_report(cnn_metrics, mlp_metrics, dataset_info)